In [1]:
import awkward as ak
import dask_awkward as dak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

/uscms/home/jlawless/nobackup/miniforge3/lib/python3.10/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [23]:
class ScoutingNanoAODSchema(NanoAODSchema):
    """ScoutingNano schema builder

    ScoutingNano is a NanoAOD format that includes Scouting objects
    """

    mixins = {
        **NanoAODSchema.mixins,
        "ScoutingPFJet": "Jet",
        "ScoutingJet": "Jet",
        "ScoutingFatJet": "Jet",
        "ScoutingMuonNoVtxDisplacedVertex": "Vertex",
        "ScoutingMuonVtxDisplacedVertex": "Vertex",
        "ScoutingPrimaryVertex":"Vertex",
        "ScoutingElectron": "Electron",
        "ScoutingPhoton": "Photon",
        "ScoutingMuonNoVtx": "Muon",
        "ScoutingMuonVtx": "Muon",

    }
    all_cross_references = {
        **NanoAODSchema.all_cross_references,
    }

In [24]:
#file_list = [s.strip("\n") for s in open("filelists/10jets.txt")]

events = NanoEventsFactory.from_root(
    dict.fromkeys(["L1Nano_wScores_1.root"], "Events"),
    schemaclass=ScoutingNanoAODSchema,
).events()

events = events[:40]

/uscms/home/jlawless/nobackup/miniforge3/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for Electron_genPartIdx => GenPart
  warnings.warn(
/uscms/home/jlawless/nobackup/miniforge3/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for Electron_jetIdx => Jet
  warnings.warn(
/uscms/home/jlawless/nobackup/miniforge3/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for Electron_photonIdx => Photon
  warnings.warn(
/uscms/home/jlawless/nobackup/miniforge3/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/uscms/home/jlawless/nobackup/miniforge3/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for L

In [4]:
events.ScoutingPFJet.fields

['HFEMMultiplicity',
 'HFHadronMultiplicity',
 'chargedHadronMultiplicity',
 'electronMultiplicity',
 'muonMultiplicity',
 'neutralHadronMultiplicity',
 'photonMultiplicity',
 'HFEMEnergy',
 'HFHadronEnergy',
 'HOEnergy',
 'chargedHadronEnergy',
 'electronEnergy',
 'eta',
 'jetArea',
 'm',
 'muonEnergy',
 'neutralHadronEnergy',
 'phi',
 'photonEnergy',
 'pt']

In [3]:
events.GenPart.pdgId.compute()

<Array [[2212, 2212, 21, ..., 111, 310], ...] type='40 * var * int32[parame...'>

In [25]:
def tight_jets(ev, jet_eta_cut: float = 2.4, jet_pt_cut: float = 30):
    jet_cut = (
        (abs(ev.ScoutingPFJet.eta) < jet_eta_cut)
        & (ev.ScoutingPFJet.pt > jet_pt_cut)
        # & (ev.ScoutingPFJet.neHEF < 0.90)
        # & (ev.ScoutingPFJet.neEmEF < 0.90)
        # & (ev.ScoutingPFJet.nConstituents > 1)
        # & (ev.ScoutingPFJet.muEF < 0.80)
        # & (ev.ScoutingPFJet.chHEF > 0.01)
        # & (ev.ScoutingPFJet.nCh > 0) \
        # & (ev.ScoutingPFJet.chEmEF < 0.80)
    )

    # gp = ak.Array({"genPartIdxMother": dak.Scalar(-999,name="genPartIdxMother")})


    # gp.pdgId = -999
    # gp.genPartIdxMother = -1

    tj = ev.ScoutingPFJet[jet_cut]
    ev["GenPart","signal"] = 0
    gp = ev.GenPart[(ev.GenPart.hasFlags(["isLastCopy"]))]

    b_one = gp[gp.pdgId == 5]
    b_one["signal"] = 1
    w_one = gp[gp.pdgId == 24].distinctChildren
    j_one = ak.flatten(w_one[w_one.hasFlags(["isLastCopy"])],axis=2)
    j_one["signal"] = 1

    b_two = gp[gp.pdgId == -5]
    b_two["signal"] = 2
    w_two = gp[gp.pdgId == -24].distinctChildren
    j_two = ak.flatten(w_two[w_two.hasFlags(["isLastCopy"])],axis=2)
    j_two["signal"] = 2

    chs = ak.concatenate([b_one, j_one, b_two, j_two], axis=1)
    ev["SlimGenPart"] = gp

    tj["MatchedGenPart"] = tj.nearest(chs,threshold=0.4)

    tj["OGGenPart"] = tj.nearest(gp,threshold=0.4)
    tj["signal"] = tj.MatchedGenPart.signal
    myslice = (tj.signal == 1) | (tj.signal == 2)
    ev["TightJet"] = tj[ak.fill_none(myslice,False)]



In [7]:
gp = events.GenPart[(events.GenPart.hasFlags(["isLastCopy"]))]

top_one = gp[gp.pdgId == 24].distinctChildren
print(top_one[top_one.hasFlags(["isLastCopy"])].pdgId.compute())

[[[-1, 2]], [[4, -3]], [[4, -3]], [[...]], ..., [[4, -3]], [[2, -1]], [[4, -3]]]


In [26]:
tight_jets(events)

In [ ]:
tj.nearest(chs,threshold=0.4)

In [20]:
chs.pdgId.compute()

<Array [[5, -1, 2, -5, 1, -2], ..., [5, ...]] type='40 * var * ?int32[param...'>

In [73]:
tj.nearest(chs,threshold=0.4)

AttributeError: nearest not in fields.

In [51]:
slg.distinctChildren.pdgId.compute()

<Array [[[24, 5, 5, 24, 5, 5, 5, 5]], ...] type='1000 * var * var * ?int32[...'>

In [23]:
events.TightJet.signal.compute()

<Array [[1, 1, 2, 1, 2, 1, 1], [...], ..., [1, 2, 1]] type='20 * var * ?int64'>

In [16]:
events.TightJet.OGGenPart.pdgId.compute()

<Array [[111, 1, 21, 21, 4, 3, 21], ..., [...]] type='20 * var * ?int32[par...'>

In [191]:
events.TightJet.OGGenPart.distinctParent.pdgId.compute()

<Array [[2, -1000005, 4, ..., 1000021, 1], ...] type='20 * var * ?int32[par...'>

In [176]:
events.TightJet.OGGenPart.distinctParent.distinctParent.pdgId.compute()

<Array [[-1000005, ..., -1000005], ..., [...]] type='20 * var * ?int32[para...'>

In [192]:
events.TightJet.OGGenPart.distinctParent.distinctParent.distinctParent.pdgId.compute()

<Array [[1000021, 21, ..., None, 1000021], ...] type='20 * var * ?int32[par...'>

In [6]:
arr = dak.zip([events.TightJet.MatchedGenPart.pdgId, events.TightJet.MatchedGenPart.distinctParent.pdgId, events.TightJet.MatchedGenPart.distinctParent.distinctParent.pdgId, events.TightJet.MatchedGenPart.distinctParent.distinctParent.distinctParent.pdgId])

In [15]:
ak.fill_none(ak.full_like(arr,1000021),100021).compute()

<Array [[(1000021, 1000021, ...), ...], ...] type='20 * var * (int64, int64...'>

In [ ]:
ak.broadcast_arrays(5, arr).compute()

AttributeError: 'int' object has no attribute 'npartitions'

In [50]:
arr.compute() == ak.fill_none(ak.full_like(arr,1000021),100021).compute()

ValueError: cannot broadcast records in equal

In [ ]:
ak.any(arr[:,:] == ak.full_like(arr,1000021))

ValueError: cannot broadcast records in equal

In [31]:
arr = ak.zip({"p0": events.TightJet.MatchedGenPart.pdgId, "p1": events.TightJet.MatchedGenPart.distinctParent.pdgId, "p2": events.TightJet.MatchedGenPart.distinctParent.distinctParent.pdgId, "p3": events.TightJet.MatchedGenPart.distinctParent.distinctParent.distinctParent.pdgId}).compute()
arr

<Array [[{p0: 111, p1: 2, ...}, ...], ...] type='20 * var * {p0: ?int32[par...'>

In [32]:
arr[0]

<Array [{p0: 111, p1: 2, p2: ..., ...}, ...] type='7 * {p0: ?int32[paramete...'>

In [26]:
arr[0]

<Array [{p1: 2, p2: -1000005}, ..., {...}] type='7 * {p1: ?int32[parameters...'>

In [ ]:
ak.num(events.TightJet).compute()

<Array [7, 10, 7, 7, 7, 6, 6, 7, ..., 7, 6, 6, 6, 7, 6, 5, 3] type='20 * int64'>

In [76]:
dir(events.TightJet.MatchedGenPart[1,:].compute())

['E',
 'E2',
 'Et',
 'Et2',
 'FLAGS',
 'M',
 'M2',
 'Mask',
 'MomentumClass',
 'Mt',
 'Mt2',
 'ProjectionClass2D',
 'ProjectionClass3D',
 'ProjectionClass4D',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__self_class__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__thisclass__',
 '_attrs',
 '_behavior',
 '_cpp_type',
 '_layout',
 '_numbaview',
 'absolute',
 'add',
 'attrs',
 'azimuthal',
 'behavior',
 'beta',
 'boost',
 'boostCM_of',
 'boostCM_of_beta3',
 'boostCM_of_p4',
 'boostX',
 'boostY',
 'boostZ',
 'boost_beta3',
 'boost_p4',
 'boostvec',
 'children',
 'childrenIdxG',
 'costheta',
 'cottheta',
 'cpp_type',
 'cross',
 'deltaR',
 'deltaR2',
 'deltaRapidityPhi',
 'deltaRapidityPhi2',
 'delta_phi',
 'del

In [78]:
events.TightJet.MatchedGenPart[1,:].distinctParentIdxG.compute()

<Array [100, 104, 104, 107, 108, -1, -1, 106, 99, -1] type='10 * ?int64'>

In [74]:
events.TightJet.MatchedGenPart[1,:].parent.compute()

<GenParticleArray [GenParticle, ..., None] type='10 * ?GenParticle[genPartI...'>

In [29]:
gp = events.GenPart[0,0].copy()


gp.pdgId = -999
gp.genPartIdxMother = -1

AttributeError: copy not in fields.

In [ ]:
ak.fill_none(events.Jet.nearest(events.SlimGenPart, threshold=0.4),gp,allow_unknown=True)

In [7]:
gp.pdgId

-999

In [22]:
events.Jet.nearest(events.SlimGenPart, threshold=0.4).compute()

<GenParticleArray [[GenParticle, GenParticle, ..., GenParticle, None], ...] type='...'>

In [30]:
ak.num(tj).compute()

<Array [7, 10, 7, 7, 7, 6, 6, 7, ..., 7, 6, 6, 6, 7, 6, 5, 3] type='20 * int64'>

In [31]:
(ak.num(events.Jet) - ak.num(tj)).compute()

<Array [1, 1, 3, 4, 2, 7, 3, 2, ..., 7, 1, 3, 3, 7, 2, 1, 3] type='20 * int64'>

In [9]:
events.GenPart.pdgId.compute()

<Array [[21, 21, 1000021, ..., 11, -11], ...] type='20 * var * int32[parame...'>

In [8]:
events.GenPart.parent.pdgId.compute()

<Array [[None, None, 21, ..., 111, 2, 2], ...] type='20 * var * ?int32[para...'>